In [10]:
import yaml

# Dependency Graph


In [11]:
def make_node(name: str, namespace: str, nominal_response_time: int) -> dict:
    return {
        "functionName": name,
        "functionNamespace": namespace,
        "nominalResponseTime": f"{nominal_response_time}m",
        "invocations": [],
    }


def add_invocation(
    node: dict, invoked_node: dict, edge_id: int, edge_multiplier: int
) -> dict:
    if "invocations" not in node:
        node["invocations"] = []
    node["invocations"].append(
        {
            "functionName": invoked_node["functionName"],
            "functionNamespace": invoked_node["functionNamespace"],
            "edgeId": edge_id,
            "edgeMultiplier": edge_multiplier,
        }
    )
    return node

In [12]:
def make_graph(name: str, namespace: str, controller_name: str) -> dict:
    return {
        "apiVersion": "neptuneplus.polimi.it/v1alpha1",
        "kind": "DependencyGraph",
        "metadata": {
            "labels": {
                "app.kubernetes.io/name": controller_name,
                "app.kubernetes.io/managed-by": "kustomize",
            },
            "name": name,
            "namespace": namespace,
        },
        "spec": {"nodes": []},
    }


def add_node(graph: dict, node: dict) -> dict:
    if "spec" not in graph:
        graph["spec"] = {"nodes": []}
    if "nodes" not in graph["spec"]:
        graph["spec"]["nodes"] = []
    graph["spec"]["nodes"].append(node)
    return graph


def save_graph(graph: dict, filename: str) -> None:
    with open(filename, "w") as f:
        yaml.dump(graph, f)

# Function / Service / SLA


In [30]:
def make_function(name: str, namespace: str, image_name: str) -> dict:
    return {
        "apiVersion": "openfaas.com/v1",
        "kind": "Function",
        "metadata": {"name": name, "namespace": namespace},
        "spec": {
            "image": image_name,
            "labels": {
                "com.openfaas.scale.factor": "20",
                "com.openfaas.scale.max": "100",
                "com.openfaas.scale.min": "1",
                "com.openfaas.scale.zero": "false",
                "edgeautoscaler.polimi.it/scheduler": "edge-autoscaler",
            },
            "name": name,
            "readOnlyRootFilesystem": False,
            "limits": {"cpu": "200m", "memory": "1000Mi"},
            "requests": {"cpu": "200m", "memory": "1000Mi"},
        },
    }


def make_service(name: str, namespace: str) -> dict:
    return {
        "apiVersion": "v1",
        "kind": "Service",
        "metadata": {
            "name": name,
            "namespace": namespace,
            "labels": {"name": name, "namespace": namespace},
        },
        "spec": {
            "ports": [{"port": 80, "targetPort": 8080}],
            "selector": {
                "edgeautoscaler.polimi.it/function-name": name,
                "edgeautoscaler.polimi.it/function-namespace": namespace,
            },
        },
    }


def make_service_level_agreement(name: str, namespace: str) -> dict:
    return {
        "apiVersion": "systemautoscaler.polimi.it/v1beta1",
        "kind": "ServiceLevelAgreement",
        "metadata": {
            "name": name,
            "namespace": namespace,
        },
        "spec": {
            "metric": {
                "responseTime": "100m",
            },
            "defaultResources": {"cpu": "300m", "memory": "768Mi"},
            "minResources": {"memory": "10Mi", "cpu": "300m"},
            "maxResources": {
                "memory": "2048Mi",
                "cpu": "20000m",
            },
            "recommenderLogic": " dependencyAware",
            "service": {
                "container": name,
                "selector": {"matchLabels": {"name": name}},
            },
        },
    }

In [31]:
def save_full_function_file(
    function: dict, service: dict, service_level_agreement: dict, filename: str
) -> None:
    function_string = yaml.dump(function, None)
    service_string = yaml.dump(service, None)
    service_level_agreement_string = yaml.dump(service_level_agreement, None)
    lines = [
        function_string,
        "---\n",
        service_string,
        "---\n",
        service_level_agreement_string,
    ]
    with open(filename, "w") as f:
        f.writelines(lines)

# EXEC


In [ ]:
node_a = make_node("function-a", "openfaas-fn", 50)
function_a = make_function(
    "function-a", "openfaas-fn", "itspeetah/npta-simple-calls-fn-a:latest"
)
service_a = make_service("function-a", "openfaas-fn")
sla_a = make_service_level_agreement("function-a", "openfaas-fn")

node_b = make_node("function-b", "openfaas-fn", 50)
function_b = make_function(
    "function-b", "openfaas-fn", "itspeetah/npta-simple-calls-fn-b:latest"
)
service_b = make_service("function-b", "openfaas-fn")
sla_b = make_service_level_agreement("function-b", "openfaas-fn")

node_c = make_node("function-c", "openfaas-fn", 50)
function_c = make_function(
    "function-c", "openfaas-fn", "itspeetah/npta-simple-calls-fn-c:latest"
)
service_c = make_service("function-c", "openfaas-fn")
sla_c = make_service_level_agreement("function-c", "openfaas-fn")

node_d = make_node("function-d", "openfaas-fn", 50)
function_d = make_function(
    "function-d", "openfaas-fn", "itspeetah/npta-simple-calls-fn-d:latest"
)
service_d = make_service("function-d", "openfaas-fn")
sla_d = make_service_level_agreement("function-d", "openfaas-fn")

node_w2 = make_node("function-w2", "openfaas-fn", 50)
function_w2 = make_function(
    "function-w2", "openfaas-fn", "itspeetah/npta-simple-calls-fn-w2:latest"
)
service_w2 = make_service("function-w2", "openfaas-fn")
sla_w2 = make_service_level_agreement("function-w2", "openfaas-fn")

node_rw = make_node("function-rw", "openfaas-fn", 50)
function_rw = make_function(
    "function-rw", "openfaas-fn", "itspeetah/npta-simple-calls-fn-rw:latest"
)
service_rw = make_service("function-rw", "openfaas-fn")
sla_rw = make_service_level_agreement("function-rw", "openfaas-fn")

graph = make_graph("depdag-npta-simple-calls", "openfaas-fn", "depdag-controller")

add_invocation(node_a, node_w2, 1, 1)
add_invocation(node_a, node_rw, 2, 1)
add_invocation(node_b, node_rw, 3, 1)
add_invocation(node_b, node_w2, 3, 1)
add_invocation(node_c, node_d, 4, 1)
add_invocation(node_c, node_w2, 4, 1)
add_invocation(node_d, node_rw, 5, 1)
add_invocation(node_d, node_rw, 5, 1)
add_invocation(node_d, node_w2, 6, 1)

add_node(graph, node_a)
add_node(graph, node_b)
add_node(graph, node_c)
add_node(graph, node_d)
add_node(graph, node_w2)
add_node(graph, node_rw)

save_graph(graph, "graph-depdag.yaml")
save_full_function_file(function_a, service_a, sla_a, "function-a.yaml")
save_full_function_file(function_b, service_b, sla_b, "function-b.yaml")
save_full_function_file(function_c, service_c, sla_c, "function-c.yaml")
save_full_function_file(function_d, service_d, sla_d, "function-d.yaml")
save_full_function_file(function_w2, service_w2, sla_w2, "function-w2.yaml")
save_full_function_file(function_rw, service_rw, sla_rw, "function-rw.yaml")